In [2]:
import argparse
import sys
import xylib

__version__ = xylib.xylib_get_version()

In [3]:
n = 0
while True:
    form = xylib.xylib_get_format(n)
    if not form:
        break
    print('%-20s: %s' % (form.name, form.desc))
    n += 1

cpi                 : Sietronics Sieray CPI
uxd                 : Bruker Diffrac-AT UXD
rigaku_dat          : Rigaku DAT
bruker_raw          : Siemens/Bruker RAW
bruker_spc          : Bruker ESP300-E SPC
vamas               : VAMAS ISO-14976
philips_udf         : Philips UDF
spe                 : PI WinSpec SPE
pdcif               : Powder Diffraction CIF
philips_rd          : Philips PC-APD RD/SD
xrdml               : PANalytical XRDML
canberra_mca        : Canberra MCA
canberra_cnf        : Canberra CNF
xfit_xdd            : XFIT XDD
riet7               : RIET7/LHPM/PSI_DMC
dbws                : DBWS data
chiplot             : ChiPLOT data
spectra             : Spectra / VGX 900
specsxy             : SPECS SpecsLab2 xy
csv                 : CSV or TSV
xsyg                : Freiberg Instruments (FI) Lexsyg
text                : Text


In [6]:
fi = xylib.xylib_get_format_by_name("bruker_raw")
if fi:
    print("Name: %s" % fi.name)
    print("Description: %s" % fi.desc)
    print("Possible extensions: %s" % (fi.exts or "(not specified)"))
    print("Flags: %s-file %s-block" % (
        ("binary" if fi.binary else "text"),
        ("multi" if fi.multiblock else "single")))
    print("Options: %s" % (fi.valid_options or '-'))
else:
    print("Unknown file format.")

Name: bruker_raw
Description: Siemens/Bruker RAW
Possible extensions: raw
Flags: binary-file multi-block
Options: -


In [7]:
def export_metadata(f, meta):
    for i in range(meta.size()):
        key = meta.get_key(i)
        value = meta.get(key)
        f.write('# %s: %s\n' % (key, value.replace('\n', '\n#\t')))

In [11]:
d = xylib.load_file('test.raw')
f = open("final.txt","w+")
f.write('# exported by xylib from a %s file\n' % d.fi.name)

43

In [13]:
nb = d.get_block_count()
for i in range(nb):
    block = d.get_block(i)
    if nb > 1 or block.get_name():
        f.write('\n### block #%d %s\n', i, block.get_name())
    else:
        export_metadata(f, block.meta)

    ncol = block.get_column_count()
    # column 0 is pseudo-column with point indices, we skip it
    col_names = [block.get_column(k).get_name() or ('column_%d' % k)
                 for k in range(1, ncol+1)]
    f.write('# ' + '\t'.join(col_names) + '\n')
    nrow = block.get_point_count()
    for j in range(nrow):
        values = ["%.6f" % block.get_column(k).get_value(j)
                  for k in range(1, ncol+1)]
        f.write('\t'.join(values) + '\n')